In [10]:
import pandas as pd
import requests
import pymongo
import requests

In [11]:
# Read the GDP per capita data from wikipedia
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28PPP%29_per_capita')

In [12]:
# get exact data from table
gdp_per_capita = tables[2]

In [13]:
# clean data and rename the columns 
gdp_per_capita_df = gdp_per_capita.loc[gdp_per_capita.loc[:,'Rank']!= '—']
gdp_per_capita_df.columns = ['rank', 'country', 'gdp_per_capita']
gdp_per_capita_df = gdp_per_capita_df.iloc[:,1:3]
gdp_per_capita_clean_df = gdp_per_capita_df.reset_index(drop=True)
gdp_per_capita_clean_df.head()

,country,gdp_per_capita
0,Qatar,138910.0
1,Luxembourg,112045.0
2,Singapore,105689.0
3,Ireland,86988.0
4,Brunei,85011.0


In [14]:
# Read the population from wikipedia
tables = pd.read_html('https://worldpopulationreview.com/countries')
tables[0]

,Rank,Country,2020 Population,2019 Population,Growth Rate,Area (km²),2018 Density
0,1,China,1439323776,1433783686,0.39%,9706961,148/km²
1,2,India,1380004385,1366417754,0.99%,3287590,416/km²
2,3,United States,331002651,329064917,0.59%,9372610,35/km²
3,4,Indonesia,273523615,270625568,1.07%,1904569,142/km²
4,5,Pakistan,220892340,216565318,2.00%,881912,246/km²
...,...,...,...,...,...,...,...
227,228,Montserrat,4992,4989,0.06%,102,49/km²
228,229,Falkland Islands,3480,3377,3.05%,12173,0/km²
229,230,Niue,1626,1615,0.68%,260,6/km²
230,231,Tokelau,1357,1340,1.27%,12,112/km²


In [15]:
# get exact data from table
country_population_df = tables[0]
# clean data and rename the columns 
country_population_df.columns = ['rank', 'country', 'population', 'population2', 'growthrate', 'countrysize', 'pop_den']
country_population_clean_df = country_population_df.loc[:,['country', 'population', 'growthrate', 'countrysize', 'pop_den']]
country_population_clean_df.loc[:,'pop_den']= country_population_clean_df.loc[:,'pop_den'].str[:-4]
country_population_clean_df.loc[:,'growthrate'] = country_population_clean_df.loc[:,'growthrate'].str[:-1]
country_population_clean_df.loc[:,'growthrate']=country_population_clean_df.loc[:,'growthrate'].astype(float)/100
country_population_clean_df['pop_den'] = country_population_clean_df['pop_den'].str.replace(',','')
country_population_clean_df.loc[:,'pop_den']=country_population_clean_df.loc[:,'pop_den'].astype(int)
gdp_per_capita_clean_df.iloc[28, 0] = 'South Korea'
country_population_clean_df



,country,population,growthrate,countrysize,pop_den
0,China,1439323776,0.0039,9706961,148
1,India,1380004385,0.0099,3287590,416
2,United States,331002651,0.0059,9372610,35
3,Indonesia,273523615,0.0107,1904569,142
4,Pakistan,220892340,0.0200,881912,246
...,...,...,...,...,...
227,Montserrat,4992,0.0006,102,49
228,Falkland Islands,3480,0.0305,12173,0
229,Niue,1626,0.0068,260,6
230,Tokelau,1357,0.0127,12,112


In [16]:
# Read the Happiness data from wikipedia
tables = pd.read_html('https://en.wikipedia.org/wiki/World_Happiness_Report#2019_World_Happiness_Report')

In [21]:
# get exact data from table
country_happiest_df = tables[4]
# clean data and rename the columns 
country_happiest_df = country_happiest_df.loc[:, ['Country or region','Score']]
country_happiest_df.columns = ['country', 'happiestScore']
country_happiest_df.loc[country_happiest_df.loc[:, "country"] == 'United States of America']
country_happiest_df.iloc[18,0] = 'United States'

In [22]:
# merging three dataframes 
populcation_data = pd.merge(country_happiest_df, country_population_clean_df, 
             how="left", 
             on = ["country","country"])
populcation_data = pd.merge(populcation_data, gdp_per_capita_clean_df , 
             how="left", 
             on = ["country","country"])
populcation_data.head()

,country,happiestScore,population,growthrate,countrysize,pop_den,gdp_per_capita
0,Finland,7.769,5540720.0,0.0015,338424.0,16.0,49548.0
1,Denmark,7.600,5792202.0,0.0035,43094.0,134.0,55675.0
2,Norway,7.554,5421241.0,0.0079,323802.0,17.0,79638.0
3,Iceland,7.494,341243.0,0.0065,103000.0,3.0,56974.0
4,Netherlands,7.488,17134872.0,0.0022,41850.0,409.0,60299.0


In [23]:
# drop empty field 
populcation_data = populcation_data.dropna()

In [32]:
populcation_data

,country,happiestScore,population,growthrate,countrysize,pop_den,gdp_per_capita
0,Finland,7.769,5540720.0,0.0015,338424.0,16.0,49548.0
1,Denmark,7.600,5792202.0,0.0035,43094.0,134.0,55675.0
2,Norway,7.554,5421241.0,0.0079,323802.0,17.0,79638.0
3,Iceland,7.494,341243.0,0.0065,103000.0,3.0,56974.0
4,Netherlands,7.488,17134872.0,0.0022,41850.0,409.0,60299.0
...,...,...,...,...,...,...,...
151,Rwanda,3.334,12952218.0,0.0258,26338.0,479.0,2642.0
152,Tanzania,3.231,59734218.0,0.0298,945087.0,61.0,3652.0
153,Afghanistan,3.203,38928346.0,0.0233,652230.0,58.0,2182.0
154,Central African Republic,3.083,4829767.0,0.0178,622984.0,8.0,864.0


In [33]:
import sqlite3, matplotlib
from sqlalchemy import create_engine

In [37]:
engine = create_engine('sqlite:///db/worldinfo.sqlite', echo=False)
sqlite_connection = engine.connect()
sqlite_table = "worldinfo"
populcation_data.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

In [ ]:
engine.execute("SELECT * FROM worldinfo").fetchall()

[(0, 'Finland', 7.769, 5540720.0, 0.0015, 338424.0, 16.0, 49548.0),
 (1, 'Denmark', 7.6, 5792202.0, 0.0034999999999999996, 43094.0, 134.0, 55675.0),
 (2, 'Norway', 7.553999999999999, 5421241.0, 0.0079, 323802.0, 17.0, 79638.0),
 (3, 'Iceland', 7.494, 341243.0, 0.006500000000000001, 103000.0, 3.0, 56974.0),
 (4, 'Netherlands', 7.4879999999999995, 17134872.0, 0.0022, 41850.0, 409.0, 60299.0),
 (5, 'Switzerland', 7.48, 8654622.0, 0.0074, 41284.0, 208.0, 67558.0),
 (6, 'Sweden', 7.343, 10099265.0, 0.0063, 450295.0, 22.0, 55989.0),
 (7, 'New Zealand', 7.307, 4822233.0, 0.008199999999999999, 270467.0, 18.0, 42045.0),
 (8, 'Canada', 7.278, 37742154.0, 0.0089, 9984670.0, 4.0, 52144.0),
 (9, 'Austria', 7.246, 9006398.0, 0.005699999999999999, 83871.0, 107.0, 55171.0),
 (10, 'Australia', 7.228, 25499884.0, 0.0118, 7692024.0, 3.0, 54799.0),
 (11, 'Costa Rica', 7.167000000000001, 5094118.0, 0.0092, 51100.0, 99.0, 18651.0),
 (12, 'Israel', 7.138999999999999, 8655535.0, 0.016, 20770.0, 410.0, 40337.0

In [48]:
# SQLAlchemy
from sqlalchemy import create_engine
engine = create_engine("sqlite:///db/worldinfo.sqlite")

conn = engine.connect()

worldinfoDF = pd.read_sql("SELECT * FROM worldinfo", conn)

worldinfoDF

,index,country,happiestScore,population,growthrate,countrysize,pop_den,gdp_per_capita
0,0,Finland,7.769,5540720.0,0.0015,338424.0,16.0,49548.0
1,1,Denmark,7.600,5792202.0,0.0035,43094.0,134.0,55675.0
2,2,Norway,7.554,5421241.0,0.0079,323802.0,17.0,79638.0
3,3,Iceland,7.494,341243.0,0.0065,103000.0,3.0,56974.0
4,4,Netherlands,7.488,17134872.0,0.0022,41850.0,409.0,60299.0
...,...,...,...,...,...,...,...,...
135,151,Rwanda,3.334,12952218.0,0.0258,26338.0,479.0,2642.0
136,152,Tanzania,3.231,59734218.0,0.0298,945087.0,61.0,3652.0
137,153,Afghanistan,3.203,38928346.0,0.0233,652230.0,58.0,2182.0
138,154,Central African Republic,3.083,4829767.0,0.0178,622984.0,8.0,864.0


In [59]:
# worldinfoDF.to_json(orient='records')

worldinfoDF.values.tolist()


[[0, 'Finland', 7.769, 5540720.0, 0.0015, 338424.0, 16.0, 49548.0],
 [1,
  'Denmark',
  7.6,
  5792202.0,
  0.0034999999999999996,
  43094.0,
  134.0,
  55675.0],
 [2, 'Norway', 7.553999999999999, 5421241.0, 0.0079, 323802.0, 17.0, 79638.0],
 [3, 'Iceland', 7.494, 341243.0, 0.006500000000000001, 103000.0, 3.0, 56974.0],
 [4,
  'Netherlands',
  7.4879999999999995,
  17134872.0,
  0.0022,
  41850.0,
  409.0,
  60299.0],
 [5, 'Switzerland', 7.48, 8654622.0, 0.0074, 41284.0, 208.0, 67558.0],
 [6, 'Sweden', 7.343, 10099265.0, 0.0063, 450295.0, 22.0, 55989.0],
 [7,
  'New Zealand',
  7.307,
  4822233.0,
  0.008199999999999999,
  270467.0,
  18.0,
  42045.0],
 [8, 'Canada', 7.278, 37742154.0, 0.0089, 9984670.0, 4.0, 52144.0],
 [9,
  'Austria',
  7.246,
  9006398.0,
  0.005699999999999999,
  83871.0,
  107.0,
  55171.0],
 [10, 'Australia', 7.228, 25499884.0, 0.0118, 7692024.0, 3.0, 54799.0],
 [11,
  'Costa Rica',
  7.167000000000001,
  5094118.0,
  0.0092,
  51100.0,
  99.0,
  18651.0],
 [12, 

In [437]:
# create json list
populcation_data_list = [];
for index, row in populcation_data.iterrows():
    populcation_dict = {
        "country":row["country"],
        "gdp_per_capita":row["gdp_per_capita"],
        "population":row["population"],
        "growthrate":row["growthrate"],
        "countrysize":row["countrysize"],
        "pop_den":row["pop_den"],
        "happiestScore":row["happiestScore"]
    }
    populcation_data_list.append(populcation_dict)
    
populcation_data_list    

[{'country': 'Finland',
  'gdp_per_capita': 49548.0,
  'population': 5540720.0,
  'growthrate': 0.0015,
  'countrysize': 338424.0,
  'pop_den': 16.0,
  'happiestScore': 7.769},
 {'country': 'Denmark',
  'gdp_per_capita': 55675.0,
  'population': 5792202.0,
  'growthrate': 0.0034999999999999996,
  'countrysize': 43094.0,
  'pop_den': 134.0,
  'happiestScore': 7.6},
 {'country': 'Norway',
  'gdp_per_capita': 79638.0,
  'population': 5421241.0,
  'growthrate': 0.0079,
  'countrysize': 323802.0,
  'pop_den': 17.0,
  'happiestScore': 7.553999999999999},
 {'country': 'Iceland',
  'gdp_per_capita': 56974.0,
  'population': 341243.0,
  'growthrate': 0.006500000000000001,
  'countrysize': 103000.0,
  'pop_den': 3.0,
  'happiestScore': 7.494},
 {'country': 'Netherlands',
  'gdp_per_capita': 60299.0,
  'population': 17134872.0,
  'growthrate': 0.0022,
  'countrysize': 41850.0,
  'pop_den': 409.0,
  'happiestScore': 7.4879999999999995},
 {'country': 'Switzerland',
  'gdp_per_capita': 67558.0,
  'p

In [416]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
client = pymongo.MongoClient('mongodb://localhost:27017')
#define the database name
mydb = client["populcation_data_DB"]
#define the collection name
mycol = mydb["populcation_data"] 
mycol.insert_many(populcation_data_list)

In [439]:
populcation_data[populcation_data['country'] == 'South Korea']


,country,happiestScore,population,growthrate,countrysize,pop_den,gdp_per_capita
53,South Korea,5.895,51269185.0,0.0009,100210.0,511.0,46452.0
